# 6. Nested Cross-Validation

**Instructions:**
* go through the notebook and complete the **tasks** .  
* Make sure you understand the examples given!
* When a question allows a free-form answer (e.g., ``what do you observe?``) create a new markdown cell below and answer the question in the notebook.
* ** Save your notebooks when you are done! **

In the previous lab, we looked at cross-validation when the parameters of our classifier (e.g., k-NN) where known.
In this lab, you will be extending the code for cross-validation in order to find the best parameters to use for each fold, by using a validation set.  Please have a look at the relevant lecture slides that demo how to apply nested cross-validation in order to remember the procedure.

**Note** You can always copy the code in a separate notebook (or, a plain text file .py that you can run with python from the command line) if you want.  After you are done, you can copy the code back in this notepad.

<hr>
<span style="color:rgb(170,0,0)">**Task:**</span> Run the cell below to load our data. Note that besides adding noise, we also initialize the numpy random seed - this is in order to always get the same results regardless of how many times we run the code. Otherwise, this piece of code is the same as the previous lab.

In [192]:
%matplotlib inline

from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt

#import k-nn classifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

iris = datasets.load_iris()

#view a description of the dataset (uncomment next line to do so)
#print(iris.DESCR)

#Set X equal to features, Y equal to the targets

X=iris.data 
y=iris.target 


mySeed=1234567
#initialize random seed generator 
np.random.seed(mySeed)

#we add some random noise to our data to make the task more challenging
X=X+np.random.normal(0,0.5,X.shape)

<hr>
<span style="color:rgb(170,0,0)">**Task:**</span> Your task is now to write your own nested cross-validation function.

You can assume that we want to run 5-fold cross-validation, and evaluate the number of neighbours (from 1 to 10 inclusive), along with the 'euclidean' and 'manhattan' distances.

Your function should split the data (using indexes) into appropriate bins, similarly to how this was done in the previous lab. 

For each fold, the testing set should consist of indices in one bin, the validation set should consist of indices in another bin, and the rest of the bins can be assigned to your training set.

Subsequently, we loop through all different parameters (one for loop for neighbours, one for loop for distances), train on the training set and test on the validation set.

Once we are done, we have the best performing set of parameters on our validation set.  We subsequently merge the training set with the validation set, and train on that set using best parameters.

Finally, we evaluate on our test set, and proceed to the next fold.

Your function should return the accuracies on the test set (with best parameters) over all five folds, e.g. ``[0.80000000000000004, 0.8666666666666667, 0.80000000000000004, 0.96666666666666667, 0.73333333333333328]``

The code below is commented so that you can work through developing the function - if you feel more comfortable, you can start working on this code in a different cell/ide and then copy the code here.

In [198]:
# nested cross validation
# X - data / features
# y - outputs
# foldK - number of folds
# nns - list of number of neighbours parameter for validation
# dists - list of distances for validation
dists=["euclidian", "manhattan"]
# mySeed - random seed
# returns: accuracy over 5 folds (list)


def myNestedCrossVal(X,y,foldK,nns,dists,mySeed):
    np.random.seed(mySeed)
    accuracy_fold=[]

    #TASK: use the function np.random.permutation to generate a list of shuffled indices from in the range (0,number of data)
    #(you did this already in a task above)
    L=list(range(X.shape[0]))
    indices = np.random.permutation(L)
    #TASK: use the function array_split to split the indices to foldK different bins (here, 5)
    #uncomment line below
    bins=np.array_split(indices, foldK)
    #print(bins)
    #no need to worry about this, just checking that everything is OK
    assert(foldK==len(bins))
    #loop through folds
    for i in range(0,foldK):
        foldTrain=[] # list to save current indices for training
        foldTest=[]  # list to save current indices for testing
        foldVal=[]    # list to save current indices for validation
        #loop through all bins, take bin i for testing, the next bin for validation, and the rest for training
        valBin = (i+1)%foldK
        for j in range(0,len(bins)):
            if (i == j):
                foldTest = bins[i]
            elif (j == valBin):
                foldVal = bins[valBin]
            else:
                foldTrain.extend(bins[j])

        #print('** Train', len(foldTrain), foldTrain)
        #print('** Val', len(foldVal), foldVal)
        #print('** Test', len(foldTest), foldTest)
        #no need to worry about this, just checking that everything is OK
        assert not np.intersect1d(foldTest,foldVal)
        assert not np.intersect1d(foldTrain,foldTest)
        assert not np.intersect1d(foldTrain,foldVal)
        bestDistance='' #save the best distance metric here
        bestNN=-1 #save the best number of neighbours here
        bestAccuracy=-10 #save the best attained accuracy here (in terms of validation)
        # loop through all parameters (one for loop for distances, one for loop for nn)
        # train the classifier on current number of neighbours/distance
        # obtain results on validation set
        # save parameters if results are the best we had
        for d in dists:
            for nn in nns:
                #split to train and test
                #define knn classifier, with 5 neighbors and use the euclidian distance
                knn=KNeighborsClassifier(n_neighbors=nn, metric=d)
                knn.fit(X[foldTrain],y[foldTrain])
                y_pred=knn.predict(X[foldVal])
                accuracy = accuracy_score(y[foldVal], y_pred)
                if accuracy > bestAccuracy:
                    bestDistance = d
                    bestNN = nn
                    bestAccuracy = accuracy

        #print('** End of val for this fold, best NN: ', bestNN, 'best Dist: ', bestDistance, '. Accuracy: ', bestAccuracy)
        #evaluate on test data:
        #extend your training set by including the validation set
        #train k-NN classifier on new training set and test on test set
        #get performance on fold, save result in accuracy_fold
        foldTrain.extend(foldVal)
        knn.fit(X[foldTrain],y[foldTrain])
        y_=knn.predict(X[foldTest])
        accuracy_=accuracy_score(y[foldTest],y_)
        accuracy_fold.append(accuracy)
        print('==== Final Cross-val on test on this fold with NN', bestNN, 'dist', bestDistance, ' accuracy ',accuracy_score(y[foldTest],y_))

    return accuracy_fold;

#call your nested crossvalidation function:
accuracy_fold=myNestedCrossVal(X,y,5,list(range(1,11)),['euclidean','manhattan'],mySeed)
print(accuracy_fold)

==== Final Cross-val on test on this fold with NN 1 dist euclidean  accuracy  0.8666666666666667
==== Final Cross-val on test on this fold with NN 9 dist euclidean  accuracy  0.8333333333333334
==== Final Cross-val on test on this fold with NN 2 dist euclidean  accuracy  0.8666666666666667
==== Final Cross-val on test on this fold with NN 5 dist euclidean  accuracy  0.9333333333333333
==== Final Cross-val on test on this fold with NN 8 dist euclidean  accuracy  0.8666666666666667
[0.8333333333333334, 0.8666666666666667, 0.9666666666666667, 0.8333333333333334, 0.8666666666666667]


/home/locua/.virtualenvs/ml1/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/locua/.virtualenvs/ml1/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/locua/.virtualenvs/ml1/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/home/locua/.virtualenvs/ml1/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wil

In [197]:
import numpy as np
foldK = 5
x = np.arange(25)
print(x)
print("\n")
#print(x_split)
for i in range(0,5):
    foldTest = x_split[i]
    foldVal = x_split[(i+1)%k]
    others = x_split[(i+2)%k:]
    foldTrain = np.concatenate(others)
    print(len(foldTest) + len(foldTrain) + len(foldVal))
    print(foldTest, foldVal, foldTrain)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]


25
[0 1 2 3 4] [5 6 7 8 9] [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
20
[5 6 7 8 9] [10 11 12 13 14] [15 16 17 18 19 20 21 22 23 24]
15
[10 11 12 13 14] [15 16 17 18 19] [20 21 22 23 24]
35
[15 16 17 18 19] [20 21 22 23 24] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
30
[20 21 22 23 24] [0 1 2 3 4] [ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]
